In [1]:
# importing libraries
import random
import json
import pickle #for serialization
import numpy as np
import pandas as pd
import nltk # natural languate toolkit
from nltk.stem import WordNetLemmatizer # reduuce the word to it's stem
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential # importing sequential model
from tensorflow.keras.layers import Dense, Activation, Dropout # importing the layers
from tensorflow.keras.optimizers import SGD # stochastic gradient descent

In [2]:
# Training
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Training
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Training
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read()) #reading the content of the json file as text

words = [] # list of all the words that we are goinf to have
classes = [] # the classes that we are goint to have
documents = [] # these are going to be the combinations
ignore_letters = ['?','!','.',','] # the letters that we are not goint to take care of

for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern) #tokenize the sentence
    words.extend(word_list)
    documents.append((word_list, intent['tag'])) # appending the tuple
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words)) # eliminating the duplicates

classes = sorted(set(classes))

# saving into files - runtime
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Machine Learning Part
training = []
output_empty = [0] * len(classes) # template of 0's and we need as many zeros as many classes

# Preparing the training data
for document in documents:
  bag = [] # for each of the combination we are creating an empty bag of words
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0) #for each word we want to know if it occurs in the patterns: 1 if the word occurs in pattern otherwise 0 

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1 # setting index to the output row as 1
  training.append([bag, output_row]) # append the whole thing to the training list

# final step of pre processing
random.shuffle(training)
training = np.array(training) #converting the traning data into numpy data

# below are the features and the labels that we are going to train our neural network
train_x = list(training[:,0])
train_y = list(training[:,1])

# building the models
model = Sequential()
# adding layers
model.add(Dense(128, input_shape=(len(train_x[0]),), activation = 'relu')) #dense layer with 128 neurons
model.add(Dropout(0.5)) #preventing overfitting
model.add(Dense(64, activation='relu')) #dense layer with 64 neurons 
model.add(Dropout(0.5)) #preventing overfitting
model.add(Dense(len(train_y[0]), activation = 'softmax')) # as many neurons as there are calsses

# Defining the stochastic gradient descent optimizer
sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbotmodel.h5', hist)
print('Done')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
7/7 [==============================] - 1s 3ms/step - loss: 1.7557 - accuracy: 0.2812
Epoch 2/200
7/7 [==============================] - 0s 2ms/step - loss: 1.7390 - accuracy: 0.2500
Epoch 3/200
7/7 [==============================] - 0s 6ms/step - loss: 1.6413 - accuracy: 0.2812
Epoch 4/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5689 - accuracy: 0.4062
Epoch 5/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4943 - accuracy: 0.3750
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5259 - accuracy: 0.3125
Epoch 7/200
7/7 [==============================] - 0s 4ms/step - loss: 1.2811 - accuracy: 0.6250
Epoch 8/200
7/7 [==============================] - 0s 3ms/step - loss: 1.3472 - accuracy: 0.5000
Epoch 9/200
7/7 [==============================] - 0s 5ms/step - loss: 1.2556 - accuracy: 0.5625
Epoch 10/200
7/7 [==============================] - 0s 3ms/step - loss: 1.2510 - accuracy: 0.5000
Epoch 11/200
7/7 [===========

In [5]:
# Main Script
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
  
  results.sort(key=lambda x: x[1], reverse = True)
  return_list = []
  for r in results:
    return_list.append({'intent':classes[r[0]], 'probability': str(r[1])})
  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

print("GO! Natalee is running!!")

while True:
  message = input("")
  ints = predict_class(message)
  res = get_response(ints, intents)
  print("Natalee: ",res)
  if (res == 'Sad to see you go.') or (res =='It was nice talking to you.') or (res =='See you again.') or (res =='So early?? Alright. BYE.') or (res =="You don't give me much time these days. Huh!! BYE!"):
    break
  else:
    continue

      
  


GO! Natalee is running!!
whats up
Natalee:  Hey!
how are you
Natalee:  I'm Natalee, here to help you.
what are you doing
Natalee:  Did you hear about the crook who stole a calendar? He got twelve months.
bye
Natalee:  You don't give me much time these days. Huh!! BYE!
